In [1]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import sys
import pandas as pd
import requests
from api_keys import AV_api, mk_token

In [2]:
# define sectors
sectors = {
    'Semiconductors': ['NVDA', 'TSM', 'AVGO', 'AMD', 'INTC'],
    'Software': ['CRM', 'SAP', 'INTU', 'NOW', 'ZI'],
    'Consumer': ['AAPL', 'MSFT', 'SONY', 'LPL', 'SONO'],
    'Infotech': ['ACN', 'IBM', 'FI', 'INFY', 'CTSH']
}
sectors = pd.DataFrame(sectors)

In [3]:
all_data = []                                                                   # create an empty list to append to

start_date = '2018-01-01'                                                       # create start and end dates

end_date = '2023-12-04'

for industry, companies in sectors.items():                                       # iterate over each industry
    for ticker in companies:                                                    # iterate over each company in industry
        try:
            print (f'Gathering data for {industry}: {ticker}')
            
            ts = TimeSeries(key=AV_api, output_format='pandas')     # make api call
            data, meta_data = ts.get_daily(symbol=ticker, outputsize='full')
            data_df = pd.DataFrame(data)

            data_df = data_df.sort_index()                                      # filter from start to end dates
            filtered_data = pd.DataFrame(data_df.loc[start_date:end_date])

            #Ticker, sector, Industry, Open, High, Low, Close, Volume
            filtered_data["Ticker"] = ticker                                    # add ticker column
            filtered_data["Industry"] = industry
            filtered_data["Sector"] = 'Technology'
            all_data.append(filtered_data)
        except:
            print(f'Company {ticker} not found')

Gathering data for Semiconductors: NVDA
Company NVDA not found
Gathering data for Semiconductors: TSM
Company TSM not found
Gathering data for Semiconductors: AVGO
Company AVGO not found
Gathering data for Semiconductors: AMD
Company AMD not found
Gathering data for Semiconductors: INTC
Company INTC not found
Gathering data for Software: CRM
Company CRM not found
Gathering data for Software: SAP
Company SAP not found
Gathering data for Software: INTU
Company INTU not found
Gathering data for Software: NOW
Company NOW not found
Gathering data for Software: ZI
Company ZI not found
Gathering data for Consumer: AAPL
Company AAPL not found
Gathering data for Consumer: MSFT
Company MSFT not found
Gathering data for Consumer: SONY
Company SONY not found
Gathering data for Consumer: LPL
Company LPL not found
Gathering data for Consumer: SONO
Company SONO not found
Gathering data for Infotech: ACN
Company ACN not found
Gathering data for Infotech: IBM
Company IBM not found
Gathering data for In

In [4]:
final_data = pd.concat(all_data)                        # Convert to dataframe
final_data.to_csv('tech_data_test.csv', index=False)        # Save to csv

In [30]:
tech_data = pd.read_csv('tech_data_test.csv')
tech_data = tech_data.rename(columns={'1. open': 'Open', 
                                      '2. high' : 'High', 
                                      '3. low' : 'Low',
                                      '4. close': 'Close',
                                      '5. volume': 'Volume',
                                      'Companies': 'sector'})
tech_data
tech_data['sector'] = 'Technology'
tech_data = tech_data[['Ticker', 'sector', 'Industry', 'Open', 'High', 'Low', 'Close', 'Volume']]
tech_data.to_csv('tech_data.csv', index = False)
tech_data

,Ticker,sector,Industry,Open,High,Low,Close,Volume
0,NVDA,Technology,Semiconductors,195.78,199.50,194.500,199.35,8303752.0
1,NVDA,Technology,Semiconductors,204.10,213.70,203.750,212.47,22499518.0
2,NVDA,Technology,Semiconductors,215.76,218.05,212.690,213.59,14346675.0
3,NVDA,Technology,Semiconductors,214.19,216.91,211.080,215.40,14008768.0
4,NVDA,Technology,Semiconductors,220.40,225.00,218.580,222.00,21190932.0
...,...,...,...,...,...,...,...,...
29059,CTSH,Technology,Infotech,70.70,71.52,70.700,71.30,2607377.0
29060,CTSH,Technology,Infotech,71.60,71.80,69.895,70.12,3521683.0
29061,CTSH,Technology,Infotech,70.39,70.52,69.800,70.38,8254394.0
29062,CTSH,Technology,Infotech,70.23,70.87,70.080,70.82,2672566.0


#### Using [Market Data API](https://docs.marketdata.app/api/)
...Because I ran out of call requests 

In [4]:
sectors = {
    'Semiconductors': ['NVDA'],
    'Software': ['CRM'],
    'Consumer': ['AAPL'],
    'Infotech': ['ACN']
}
sectors = pd.DataFrame(sectors)

In [14]:
url = 'https://api.marketdata.app/v1/markets'
token = mk_token
resolution = 'daily'
start = '2018-01-01'
end = '2023-12-04'
all_data = []
not_found = []

for industry, companies in sectors.items():                                       # iterate over each industry
    for ticker in companies:                                                    # iterate over each company in industry
        # try:
        print (f'Gathering data for {industry}: {ticker}')

        symbol = ticker
        url = f'https://api.marketdata.app/v1/stocks/candles/{resolution}/{symbol}?from={start}&to={end}&token={token}'
        response = requests.get(url).json()
        print('Response status:', response['s'])

        data = {
        'ticker': ticker,
        'date': response['t'],
        'open': response['o'],
        'close': response['c'],
        'high': response['h'],
        'low': response['l'],
        'volume': response['v']
        }

        data['date'] = pd.to_datetime(data['date'], unit='s')
        data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d')
        data = data.sort_index()  

        filtered_data = pd.DataFrame(data_df.loc[start_date:end_date])

        #Ticker, sector, Industry, Open, High, Low, Close, Volume
        filtered_data["Ticker"] = ticker                                    # add ticker column
        filtered_data["Industry"] = industry
        filtered_data["Sector"] = 'Technology'
        all_data.append(filtered_data)

        # except:
        #     print (f'{ticker} not found')
        #     not_found.append(ticker)


Gathering data for Semiconductors: NVDA
Response status: ok


AttributeError: 'dict' object has no attribute 'sort_index'

[]